In [1]:
import pandas as pd
import numpy as np

from bokeh.layouts import column
from bokeh.models import RangeTool, LinearAxis, Range1d, Legend, Label
from bokeh.plotting import figure, output_file, save

In [2]:
# results summary for the validation period
val = np.load("../results/summary_validation.npy", allow_pickle=True).item()

In [3]:
selected_basins = ['02055100', '02143000',
                   '12143600', '11381500',
                   '03500000', '14306500'
                  ]



basin_names = {'02055100': "Tinker Creek", 
               '02143000': "Henry Fork",
               '12143600': "Little Tennessee", 
               '11381500': "Mill Creek",
               '03500000': "Alsea River", 
               '14306500': "Snoqualmie River"}

In [4]:
def get_data(basin, model):
    
    fp_shortcut = '1996_2009'
    
    fp_nse = val[basin][model][14]['NSE'][fp_shortcut]
    fp_nse = np.round(fp_nse, 2)
    
    fp_bias = val[basin][model][14]['Bias'][fp_shortcut]
    fp_bias = int(np.round(fp_bias, 0))
    
    if model == "GR4H":
        fp_data = pd.read_pickle(f"../results/experiment_results/{basin}/{model}/validation_{fp_shortcut}.pkl")
    else:
        fp_data = pd.read_pickle(f"../results/experiment_results/{basin}/{model}/validation_{fp_shortcut}_20_4320.pkl")
    
    
    sy_dict = val[basin][model][1]['NSE']
    sy_dict_values = np.array(list(sy_dict.values()))
    
    sy_shortcut = list(sy_dict.keys())[np.argmin(np.abs(sy_dict_values - sy_dict_values.mean()))]
    
    sy_nse = val[basin][model][1]['NSE'][sy_shortcut]
    sy_nse = np.round(sy_nse, 2)
    
    sy_bias = val[basin][model][1]['Bias'][sy_shortcut]
    sy_bias = int(np.round(sy_bias, 0))
    
    if model == "GR4H":
        sy_data = pd.read_pickle(f"../results/experiment_results/{basin}/{model}/validation_{sy_shortcut}.pkl")
    else:
        sy_data = pd.read_pickle(f"../results/experiment_results/{basin}/{model}/validation_{sy_shortcut}_20_4320.pkl")
       
    return fp_data, sy_data, fp_nse, fp_bias, sy_nse, sy_bias, sy_shortcut.split("_")[1]

In [5]:
data_dict = {basin:{model:None} for basin in selected_basins for model in ["GR4H", "LSTM", "GRU"]}

for basin in selected_basins:
    
    for model in ["GR4H", "LSTM", "GRU"]:
        
        fp_data, sy_data, fp_nse, fp_bias, sy_nse, sy_bias, sy_shrt = get_data(basin, model)
        
        data_dict[basin][model] = {"fp_data": fp_data,
                                   "sy_data": sy_data,
                                   "fp_nse": fp_nse, 
                                   "fp_bias": fp_bias,
                                   "sy_nse": sy_nse, 
                                   "sy_bias": sy_bias,
                                   "sy_shrt": sy_shrt}

In [8]:
def get_plottable(basin_ex):    

    dates = np.array(data_dict[basin_ex]['GR4H']['fp_data'].index, dtype=np.datetime64)

    TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,undo"

    p = figure(plot_height=300, 
               plot_width=800, 
               tools=TOOLS, 
               toolbar_location="above",
               x_axis_type="datetime", 
               x_axis_location="above",
               background_fill_color="#efefef", 
               x_range=(dates[0], dates[-1]), 
               title=f"{basin_ex}: {basin_names[basin_ex]}")
    
    legend_items = []
    
    l0 = p.line(data_dict[basin_ex]["GR4H"]['fp_data'].index,
                data_dict[basin_ex]["GR4H"]['fp_data']["Qobs"],
                color="green", line_width=1.5)
    
    legend_items.append( ("Observation", [l0]) )

    color_dict = {"LSTM": "royalblue", "GRU": "firebrick", "GR4H": "black"}

    for model in ["GR4H", "LSTM", "GRU"]:

        l1 = p.line(data_dict[basin_ex][model]['fp_data'].index,
                    data_dict[basin_ex][model]['fp_data']['Qsim'],
                    color=color_dict[model], alpha=1, line_width=1.5)

        l2 = p.line(data_dict[basin_ex][model]['sy_data'].index,
                    data_dict[basin_ex][model]['sy_data']['Qsim'],
                    color=color_dict[model], alpha=0.75, line_width=1.5, line_dash="dashed")
        
        legend_items.append( (f'{model}, 14 yr ({data_dict[basin_ex][model]["fp_nse"]}/{data_dict[basin_ex][model]["fp_bias"]})', [l1]) )
        legend_items.append( (f'{model}, 1 yr ({data_dict[basin_ex][model]["sy_nse"]}/{data_dict[basin_ex][model]["sy_bias"]})', [l2]) )
    
    p.yaxis.axis_label = "Runoff, mm/hour"
     
    legend = Legend(items=legend_items, 
                    location="center", 
                    title='Click to hide the lines')
    
    p.add_layout(legend, 'right')
    p.legend.click_policy="hide"
    
    #####
    mdata = pd.read_pickle(f"../forcing/{basin_ex}.pkl")
    
    p2 = figure(plot_height=150, 
               plot_width=800, 
               tools="", 
               toolbar_location=None,
               x_axis_type="datetime", 
               x_axis_location="above",
               background_fill_color="#efefef", 
               x_range=p.x_range, 
               title="") 
    
    p2.line(mdata.index,
            mdata["T"],
            legend_label='Air temperature', color="red", line_width=1.5)
    
    p2.yaxis.axis_label = "Temperature, deg C"
    
    
    p2.extra_y_ranges = {"Prec": Range1d(start=0, end=max(mdata["P"]))}
    p2.add_layout(LinearAxis(y_range_name="Prec", axis_label='Precipitation, mm'), 'right')
    
    p2.line(mdata.index,
            mdata["P"],
            legend_label='Precipitation', color="blue", line_width=1.5, y_range_name="Prec")
    
       
    
    #####
    select = figure(title="Drag the middle and edges of the selection box to change the range above",
                    plot_height=130, plot_width=800, y_range=p.y_range,
                    x_axis_type="datetime", y_axis_type=None,
                    tools="", toolbar_location=None, background_fill_color="#efefef")

    range_tool = RangeTool(x_range=p.x_range)
    range_tool.overlay.fill_color = "navy"
    range_tool.overlay.fill_alpha = 0.2

    select.line(data_dict[basin_ex]["GR4H"]['fp_data'].index,
                data_dict[basin_ex]["GR4H"]['fp_data']["Qobs"])
    
    select.ygrid.grid_line_color = None
    select.add_tools(range_tool)
    select.toolbar.active_multi = range_tool
    
    """
    description = "Very long description of what is happening\n in the world of tanks, zombies and bies"
    
    citation = Label(x=0, y=-50, x_units='screen', y_units='screen',
                 text=description, render_mode='css',
                 border_line_color='black', border_line_alpha=0.5,
                 background_fill_color='white', background_fill_alpha=1.0)
    
    select.add_layout(citation)
    """
    
    plot = column(p, p2, select)
    
    return plot

In [9]:
for basin in selected_basins:
    
    pli = get_plottable(basin)
    output_file(f'../docs/{basin}.html', title=basin)
    save(pli)